<!-- app-mode -->

In [ ]:
%%sh
pip install scipy > /dev/null

In [ ]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib widget
import logging
import warnings

import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt
import numpy as np
import qrules
import scipy.integrate
import symplot
import sympy as sp
from ampform.dynamics import (
    BlattWeisskopfSquared,
    breakup_momentum_squared,
    coupled_width,
    phase_space_factor,
    phase_space_factor_analytic,
    phase_space_factor_complex,
    relativistic_breit_wigner,
    relativistic_breit_wigner_with_ff,
)
from ampform.dynamics.math import ComplexSqrt
from IPython.display import HTML, Math
from ipywidgets import widgets

logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

warnings.filterwarnings("ignore")

np.seterr(divide="ignore")

PDG = qrules.load_pdg()

HTML(
    """
<style>
div.prompt {display:none}
</style>
"""
)

# Analytic continuation

Following slides summarizes two attempts at respecting analyticity in the $S$-matrix:
1. The existing implementation in AmpForm following [[PDG2018, §Resonances, p.9](https://pdg.lbl.gov/2018/reviews/rpp2018-rev-resonances.pdf#page=9)]
2. A sketch of solving the dispersion integral following [[PDG2020, §Resonances, pp.9–10](https://pdg.lbl.gov/2020/reviews/rpp2020-rev-resonances.pdf#page=9)] (Chew-Mandelstam)

## 'Case-wise' phase space factor

In [ ]:
m, m0, w0, m_a, m_b, d = sp.symbols("m m0 Gamma0 m_a m_b d")
L = sp.Symbol("L", integer=True)
s = m ** 2

In [ ]:
rel_bw = relativistic_breit_wigner(
    s=s,
    mass0=m0,
    gamma0=w0,
)
rel_bw_with_ff = relativistic_breit_wigner_with_ff(
    s=s,
    mass0=m0,
    gamma0=w0,
    m_a=m_a,
    m_b=m_b,
    angular_momentum=L,
    meson_radius=d,
    phsp_factor=phase_space_factor,
)

running_width = coupled_width(
    s=s,
    mass0=m0,
    gamma0=w0,
    m_a=m_a,
    m_b=m_b,
    angular_momentum=L,
    meson_radius=d,
    phsp_factor=phase_space_factor,
)

Non-relativistic Breit-Wigner:

In [ ]:
rel_bw

Relativistic Breit-Wigner with form factor _above threshold only_ [[PDG2020, §Resonances, p.6](https://pdg.lbl.gov/2020/reviews/rpp2020-rev-resonances.pdf#page=6)]:

In [ ]:
q_squared = breakup_momentum_squared(s, m_a, m_b)
ff_sq = BlattWeisskopfSquared(L, z=q_squared * d ** 2)
rel_bw_with_ff_subs = rel_bw_with_ff.subs(
    {
        2 * q_squared: 2 * sp.Symbol("q^{2}(m)"),
        ff_sq: sp.Symbol(R"\color{blue}{B_{L}^{2}\left(q\right)}"),
        running_width: sp.Symbol(R"\Gamma(m)"),
    }
)
rel_bw_with_ff_subs

Coupled width [[PDG2020, §Resonances, Eq. (49.21)](https://pdg.lbl.gov/2020/reviews/rpp2020-rev-resonances.pdf#page=6)]:

In [ ]:
q_squared = breakup_momentum_squared(s, m_a, m_b)
q0_squared = breakup_momentum_squared(m0 ** 2, m_a, m_b)
ff_sq = BlattWeisskopfSquared(L, z=q_squared * d ** 2)
ff0_sq = BlattWeisskopfSquared(L, z=q0_squared * d ** 2)
rho = phase_space_factor(s, m_a, m_b)
rho0 = phase_space_factor(m0 ** 2, m_a, m_b)
running_width = running_width.subs(
    {
        rho
        / rho0: sp.Symbol(R"\color{red}{\rho(m)}")
        / sp.Symbol(R"\color{red}{\rho(m_{0})}"),
        ff_sq: sp.Symbol("B_{L}^{2}(q)"),
        ff0_sq: sp.Symbol("B_{L}^{2}(q_{0})"),
    },
)
running_width_math = Math(fR"\Gamma(m) = {sp.latex(running_width)}")
running_width_math

with $B_L^2(q)$ the Blatt-Weisskopf form factor as defined [here](https://ampform.readthedocs.io/en/stable/api/ampform.dynamics.html#ampform.dynamics.BlattWeisskopfSquared).

### Phase space factor

We have to choose a phase space factor $\color{red}{\rho}$ for the coupled width:

In [ ]:
running_width_math

### Phase space factor ― some options

In [ ]:
q_squared = breakup_momentum_squared(s, m_a, m_b)
rho = phase_space_factor(s, m_a, m_b)
rho_analytic = phase_space_factor_analytic(s, m_a, m_b)

**Break-up momentum** [[PDG2020, §Kinematics, Eq. (48.1)](https://pdg.lbl.gov/2020/reviews/rpp2020-rev-kinematics.pdf#page=3)]:

In [ ]:
q_squared_math = Math(f"q(m) = {sp.latex(sp.sqrt(q_squared, evaluate=False))}")
q_squared_math

'Standard' **phase space factor** [[PDG2020, §Resonances, Eq. (49.8)](https://pdg.lbl.gov/2020/reviews/rpp2020-rev-resonances.pdf#page=4)]:

In [ ]:
rho_subs = rho.subs(sp.sqrt(q_squared), sp.Symbol(R"\sqrt{q^{2}(m)}"))
rho_math = Math(fR"\rho(m) = {sp.latex(rho_subs)}")
rho_math

'Case-wise' phase space factor [[PDG2018, §Resonances, p.9](https://pdg.lbl.gov/2018/reviews/rpp2018-rev-resonances.pdf#page=9)] (<span style="color:red">only valid if $m_a = m_b$ (?)</span>):

In [ ]:
from ampform.dynamics import _analytic_continuation, phase_space_factor_abs

rho_ac_math = _analytic_continuation(
    sp.Symbol(R"\hat{\rho}"), s, s_threshold=(m_a + m_b) ** 2
)
rho_hat = phase_space_factor_abs(s, m_a, m_b)
rho_hat_subs = rho_hat.subs(
    4 * sp.Abs(q_squared),
    4 * sp.Symbol(R"\left|q^{2}(m)\right|"),
)
rho_ac_math = Math(
    sp.latex(rho_ac_math)
    + R" \quad \mathrm{{with}} \quad \hat{{\rho}}(m) = "
    + sp.latex(rho_hat_subs)
)
rho_ac_math

### Break-up momentum behavior

Underlying issue: branching solutions for <br>
$|m_a-m_b| < m < m_a+m_b$:

In [ ]:
q_squared_math

In [ ]:
np_breakup_momentum_squared, sliders = symplot.prepare_sliders(
    plot_symbol=m,
    expression=q_squared,
)


def np_breakup_momentum_squared_left(plot_domain, m_a, m_b):
    values = np_breakup_momentum_squared(plot_domain, m_a, m_b)
    return np.where(plot_domain < np.abs(m_a - m_b), values, np.nan)


def np_breakup_momentum_squared_mid(plot_domain, m_a, m_b):
    values = np_breakup_momentum_squared(plot_domain, m_a, m_b)
    return np.where(values < 0, values, np.nan)


def np_breakup_momentum_squared_right(plot_domain, m_a, m_b):
    values = np_breakup_momentum_squared(plot_domain, m_a, m_b)
    return np.where(plot_domain > m_a + m_b, values, np.nan)


plot_domain = np.linspace(0.01, 4, 400)
sliders.set_ranges(
    m_a=(0, 2, 200),
    m_b=(0, 2, 200),
)
sliders.set_values(
    m_a=1.4,
    m_b=0.9,
)

fig, ax = plt.subplots(figsize=(9, 4.3), tight_layout=True)
fig.canvas.toolbar_visible = False
ax.set_xlabel("$m$")
ax.set_ylabel("$q^2(m)$")
controls = iplt.plot(
    plot_domain,
    np_breakup_momentum_squared_left,
    **sliders,
    ylim=(-1, 3),
    ax=ax,
    label=R"$m \leq m_a-m_b$",
    c="C2",
)
iplt.plot(
    plot_domain,
    np_breakup_momentum_squared_mid,
    controls=controls,
    ylim="auto",
    ax=ax,
    label="$q^2(m) < 0$",
    c="red",
)
iplt.plot(
    plot_domain,
    np_breakup_momentum_squared_right,
    controls=controls,
    ylim="auto",
    ax=ax,
    label=R"$m \geq m_a+m_b$",
    c="C0",
)
ax.set_yticks([0])
plt.axhline(y=0, c="black", linestyle="dotted", linewidth=1)
plt.legend(loc="lower right")
plt.show()

### Analytic continuation

Analytic continuation should take care of those negative values:

In [ ]:
rho_ac_math

In the following slide, we visualize compare this continuation with slightly modified versions of $q$ and $\rho$:

In [ ]:
q_squared_symbol = sp.Symbol("q^{2}(m)")
q_c = ComplexSqrt(q_squared_symbol)
rho_c = phase_space_factor_complex(s, m_a, m_b)

x = sp.Symbol("x", real=True)
complex_sqrt = ComplexSqrt(x)

q_c_symbol = sp.Symbol("q_{c}(m)")
q_c_subs = q_c.subs(4 * q_squared, 4 * q_squared_symbol)
rho_c_subs = rho_c.subs(ComplexSqrt(q_squared), ComplexSqrt(q_squared_symbol))

q_c_latex = fR"{sp.latex(q_c_symbol)} = {sp.latex(q_c)}"
rho_c_latex = fR"\rho_c(m) = {sp.latex(rho_c_subs)}"
display(
    Math(q_c_latex),
    Math(rho_c_latex),
    Math(fR"{sp.latex(complex_sqrt)} = {sp.latex(complex_sqrt.evaluate())}"),
)

<span style="color:gray;">This is only to compare analytic behavior to _what $q$ and $\rho$ would have been_ below threshold if their square root were defined (see [`ComplexSqrt`](https://ampform.readthedocs.io/en/stable/api/ampform.dynamics.math.html#ampform.dynamics.math.ComplexSqrt)). Normally, AmpForm leaves $q$ and $\rho$ undefined below threshold.</span>

### Analytic continuation

In [ ]:
np_rho_c, sliders = symplot.prepare_sliders(plot_symbol=m, expression=rho_c)
np_rho_ac = sp.lambdify((m, m_a, m_b), rho_analytic, "numpy")
np_breakup_momentum = sp.lambdify(
    (m, m_a, m_b), ComplexSqrt(q_squared), "numpy"
)

plot_domain = np.linspace(0, 3, 200)
sliders.set_ranges(
    m_a=(0, 2, 200),
    m_b=(0, 2, 200),
)
sliders.set_values(
    m_a=0.6,
    m_b=1.25,
)

fig, axes = plt.subplots(1, 3, figsize=[13, 4], tight_layout=True)
fig.canvas.toolbar_visible = False

ax_q, ax_rho, ax_rho_ac = axes
for ax in axes:
    ax.set_xlabel("$m$")

ylim_q = (-0.04, 1.0)
ylim_rho = (-0.002, 0.05)


def func_imag(func, *args, **kwargs):
    return lambda *args, **kwargs: func(*args, **kwargs).imag


def func_real(func, *args, **kwargs):
    return lambda *args, **kwargs: func(*args, **kwargs).real


ax_q.set_title(f"${q_c_latex}$")
controls = iplt.plot(
    plot_domain,
    func_real(np_breakup_momentum),
    label="real",
    **sliders,
    ylim=ylim_q,
    ax=ax_q,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_breakup_momentum),
    label="imaginary",
    controls=controls,
    ylim=ylim_q,
    ax=ax_q,
    alpha=0.7,
)

ax_rho.set_title(f"${rho_c_latex}$")
iplt.plot(
    plot_domain,
    func_real(np_rho_c),
    label="real",
    controls=controls,
    ylim=ylim_rho,
    ax=ax_rho,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_rho_c),
    label="imaginary",
    controls=controls,
    ylim=ylim_rho,
    ax=ax_rho,
    alpha=0.7,
)

ax_rho_ac.set_title("analytic")
iplt.plot(
    plot_domain,
    func_real(np_rho_ac),
    label="real",
    controls=controls,
    ylim=ylim_rho,
    ax=ax_rho_ac,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_rho_ac),
    label="imaginary",
    controls=controls,
    ylim=ylim_rho,
    ax=ax_rho_ac,
    alpha=0.7,
)
plt.legend(loc="upper right")

plt.show()

In [ ]:
rho_ac_math

### Effect on Breit-Wigner

In [ ]:
def breakup_momentum(s: sp.Symbol, m_a: sp.Symbol, m_b: sp.Symbol) -> sp.Expr:
    q_squared = breakup_momentum_squared(s, m_a, m_b)
    return ComplexSqrt(q_squared)


bw_q = relativistic_breit_wigner_with_ff(
    s=s,
    mass0=m0,
    gamma0=w0,
    m_a=m_a,
    m_b=m_b,
    angular_momentum=L,
    meson_radius=d,
    phsp_factor=breakup_momentum,
)
bw_rho = relativistic_breit_wigner_with_ff(
    s=s,
    mass0=m0,
    gamma0=w0,
    m_a=m_a,
    m_b=m_b,
    angular_momentum=L,
    meson_radius=d,
    phsp_factor=phase_space_factor_complex,
)
bw_rho_ac = relativistic_breit_wigner_with_ff(
    s=s,
    mass0=m0,
    gamma0=w0,
    m_a=m_a,
    m_b=m_b,
    angular_momentum=L,
    meson_radius=d,
    phsp_factor=phase_space_factor_analytic,
)


def make_argand(func):
    def decorated_func(*args, **kwargs):
        values = func(plot_domain, *args, **kwargs)
        argand = np.array([values.real, values.imag])
        return argand.T

    return decorated_func

### Questions about this implementation

Analytic continuation ($\color{red}{\rho}$):
- Does analytic continuation in a Breit-Wigner <br>make sense at all?
- Are the two 'bumps' when below threshold desired?
- What happened to the 'case-wise' analytic continuation? <br>[[PDG2018, §Resonances, p.9](https://pdg.lbl.gov/2018/reviews/rpp2018-rev-resonances.pdf#page=9)]

Relativistic Breit-Wigner with form factor ($\color{blue}{B_L}$):
- What to do with the damping factor below threshold?
- With damping factor $F^2_0(z)=1, F^2_1(z)=\frac{1}{1+z}, F^2_2(z)=\frac{1}{9+3z+z^2}, ...$<br>
  should we choose $z=q/q_0$ or $z=\left(q/q_0\right)^2$?<br>
  _See note that comes with [`BlattWeisskopfSquared`](https://ampform.readthedocs.io/en/stable/api/ampform.dynamics.html#ampform.dynamics.BlattWeisskopfSquared)_

In [ ]:
np_bw_rho_ac, sliders = symplot.prepare_sliders(
    plot_symbol=m,
    expression=bw_rho_ac.doit(),
)
np_bw_rho = sp.lambdify((m, w0, L, d, m0, m_a, m_b), bw_rho.doit())
np_bw_q = sp.lambdify((m, w0, L, d, m0, m_a, m_b), bw_q.doit())

plot_domain = np.linspace(start=0, stop=4, num=501)
sliders.set_ranges(
    m0=(0, 4, 50),
    Gamma0=(0, 1, 100),
    L=(0, 8),
    m_a=(0, 2, 200),
    m_b=(0, 2, 200),
    d=(0, 5),
)


def set_defaults(ev):
    return sliders.set_values(
        m0=1.8,
        Gamma0=0.6,
        L=1,
        m_a=0.7,
        m_b=0.5,
        d=1,
    )


set_defaults(None)
starting_values = widgets.Button(description="defaults")
starting_values.on_click(set_defaults)

zero_angular_momentum = widgets.Button(description="L=0")
zero_angular_momentum.on_click(
    lambda ev: sliders.set_values(
        L=0,
    )
)

below_threshold = widgets.Button(description="below threshold")
below_threshold.on_click(
    lambda ev: sliders.set_values(
        m0=0.95,
        m_a=0.7,
        m_b=0.5,
    )
)

just_below_threshold = widgets.Button(description="just below threshold")
just_below_threshold.on_click(
    lambda ev: sliders.set_values(
        m0=1.04,
        m_a=0.7,
        m_b=0.5,
    )
)

button_box = widgets.HBox(
    [
        starting_values,
        below_threshold,
        just_below_threshold,
        zero_angular_momentum,
    ]
)

fig, all_axes = plt.subplots(
    nrows=3,
    ncols=2,
    figsize=(8, 8),
    gridspec_kw={"width_ratios": [1.8, 1]},
)
fig.suptitle("Relativistic Breit-Wigners with different phase space factors")
fig.canvas.toolbar_visible = False

for ax_left, ax_right in all_axes:
    ax_right.set_ylabel(R"Im$\left(A\right)$")
m_label = "$m$"
ax_left.set_xlabel(m_label)
ax_right.set_xlabel(R"Re$\left(A\right)$")
axes, axes_argand = all_axes.T

axes[0].set_title("'Complex' break-up momentum $q$")
axes[1].set_title(R"'Complex' phase space factor $\rho_\mathrm{c}$")
axes[2].set_title(R"'Case-wise' phase space factor")

ylim = "auto"  # (-0.3, 0.8)

controls = iplt.plot(
    plot_domain,
    lambda *args, **kwargs: np_bw_q(*args, **kwargs).real,
    label="real",
    **sliders,
    ylim=ylim,
    ax=axes[0],
)
iplt.plot(
    plot_domain.real,
    lambda *args, **kwargs: np_bw_rho(*args, **kwargs).real,
    label="real",
    controls=controls,
    ylim=ylim,
    ax=axes[1],
)
iplt.plot(
    plot_domain.real,
    lambda *args, **kwargs: np_bw_rho_ac(*args, **kwargs).real,
    label="real",
    controls=controls,
    ylim=ylim,
    ax=axes[2],
)
iplt.plot(
    plot_domain.real,
    lambda *args, **kwargs: np_bw_q(*args, **kwargs).imag,
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=axes[0],
)
iplt.plot(
    plot_domain.real,
    lambda *args, **kwargs: np_bw_rho(*args, **kwargs).imag,
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=axes[1],
)
iplt.plot(
    plot_domain.real,
    lambda *args, **kwargs: np_bw_rho_ac(*args, **kwargs).imag,
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=axes[2],
)

for ax in axes:
    iplt.axvline(
        controls["m0"],
        c="red",
        alpha=0.3,
        ax=ax,
        label="$m_0$",
    )
    iplt.axvline(
        lambda m_a, m_b, **kwargs: m_a + m_b,
        controls=controls,
        c="black",
        alpha=0.3,
        ax=ax,
        label="$m_a + m_b$",
    )
    ax.axhline(0, c="black", linewidth=1)
    if isinstance(ylim, tuple):
        ax.set_yticks([0])
axes[0].legend(loc="upper right")

# Argand plots
iplt.scatter(
    make_argand(np_bw_q),
    controls=controls,
    parametric=True,
    c=plot_domain,
    s=1,
    xlim="auto",
    ylim="auto",
    ax=axes_argand[0],
)
iplt.scatter(
    make_argand(np_bw_rho),
    controls=controls,
    parametric=True,
    c=plot_domain,
    s=1,
    xlim="auto",
    ylim="auto",
    ax=axes_argand[1],
)
iplt.scatter(
    make_argand(np_bw_rho_ac),
    controls=controls,
    parametric=True,
    c=plot_domain,
    s=1,
    xlim="auto",
    ylim="auto",
    ax=axes_argand[2],
)
for ax in axes_argand:
    ax.set_xticks([0])
    ax.set_yticks([0])
    plt.colorbar(label=m_label, ax=ax, aspect=30, pad=0.01)
plt.winter()

fig.tight_layout()
plt.show()
display(button_box)

### Formula overview

In [ ]:
display(
    rel_bw_with_ff_subs,
    running_width_math,
    rho_ac_math,
    rho_math,
    q_squared_math,
)

## Chew-Mandelstam

- The 'case-wise' phase space factor has been removed from the PDG in 2019
- Instead, it describes an approach _specific to the $K$-matrix_ [[PDG2020, §Resonances, pp.9–10](https://pdg.lbl.gov/2020/reviews/rpp2020-rev-resonances.pdf#page=9)]:
  - Chew-Mandelstam in the case of an $S$-wave
  - Dispersion integral in the case of non-zero angular momentum $L>0$

<span style="color:gray;">See also [ComPWA TR-003](https://compwa-org.readthedocs.io/en/stable/report/003.html)</span>

### $S$-wave

In [ ]:
phsp_scale_factor = 16 * sp.pi ** 2


def breakup_momentum(s, m1, m2):
    q_squared = breakup_momentum_squared(s, m1, m2)
    return ComplexSqrt(q_squared)


def chew_mandelstam_s_wave(s, m1, m2):
    q = breakup_momentum(s, m1, m2)
    left_term = sp.Mul(
        2 * q / sp.sqrt(s),
        sp.log((m1 ** 2 + m2 ** 2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2)),
        evaluate=False,
    )
    right_term = m1 ** 2 - m2 ** 2
    right_term *= 1 / s - 1 / (m1 + m2) ** 2
    right_term *= sp.log(m1 / m2)
    return (left_term - right_term) / phsp_scale_factor

For a particle $a$ decaying to particles $1, 2$ with $L=0$, the **Chew-Mandelstam function** reads:

In [ ]:
s, m1, m2 = sp.symbols("s m1 m2", real=True)
chew_mandelstam_s_wave_expr = chew_mandelstam_s_wave(s, m1, m2)

chew_mandelstam_s_wave_symbolic = chew_mandelstam_s_wave_expr.subs(
    2 * breakup_momentum(s, m1, m2),
    2 * sp.Symbol("q_a"),
)
chew_mandelstam_s_wave_symbolic *= phsp_scale_factor
chew_mandelstam_s_wave_symbolic_scaled = sp.Mul(
    1 / phsp_scale_factor,
    chew_mandelstam_s_wave_symbolic,
    evaluate=False,
)
chew_mandelstam_s_wave_symbolic_math = Math(
    fR"\Sigma_a(s)={sp.latex(chew_mandelstam_s_wave_symbolic_scaled)}"
)
chew_mandelstam_s_wave_symbolic_math

The PDG does not state how this function should be defined for $s < (m_1+m_2)^2$, but we assume:

In [ ]:
q_squared_symbol = sp.Symbol("q_a^{2}", real=True)
q_a_expr = ComplexSqrt(q_squared_symbol)
Math(f"q_a = {sp.latex(q_a_expr)} = {sp.latex(q_a_expr.evaluate())}")

Figure 49.3 [[PDG2020, §Resonances, pp.10](https://pdg.lbl.gov/2020/reviews/rpp2020-rev-resonances.pdf#page=10)] in the PDG plots this function with a **slight offset from the real axes** of $s$:

In [ ]:
epsilon = sp.Symbol("epsilon", real=True, positive=True)
s_prime = sp.Symbol(R"s^{\prime}", real=True)
s_plus = s_prime + sp.I * sp.Pow(10, -epsilon)
display(
    chew_mandelstam_s_wave_symbolic_math,
    Math(fR"{sp.latex(s)} \to {sp.latex(s_plus)}"),
)

In [ ]:
chew_mandelstam_s_wave_prime = chew_mandelstam_s_wave_expr.subs(s, s_plus)
np_chew_mandelstam_s_wave, sliders_cm = symplot.prepare_sliders(
    expression=chew_mandelstam_s_wave_prime,
    plot_symbol=s_prime,
)
np_phase_space_factor = sp.lambdify(
    args=(s_prime, m1, m2, epsilon),
    expr=phase_space_factor_complex(s_plus, m1, m2),
    modules="numpy",
)

s_min, s_max = -0.15, 1.4
m1_val = PDG["pi0"].mass
m2_val = PDG["eta"].mass

plot_domain = np.linspace(s_min, s_max, 500)
sliders_cm.set_ranges(
    m1=(0, 2, 200),
    m2=(0, 2, 200),
    epsilon=(1, 12),
)
sliders_cm.set_values(
    m1=m1_val,
    m2=m2_val,
    epsilon=4,
)

fig, axes = plt.subplots(ncols=2, figsize=(13, 5), tight_layout=True)
ax1, ax2 = axes
for ax in axes:
    ax.axhline(0, linewidth=0.5, c="black")

ylim = (-1, +1)
y_factor = 16 * np.pi
controls = iplt.axvline(
    lambda *args, **kwargs: (kwargs["m1"] + kwargs["m2"]) ** 2,
    **sliders_cm,
    c="grey",
    linewidth=0.5,
    label=R"$s_\mathrm{thr}$",
    ax=ax1,
)
iplt.axvline(
    lambda *args, **kwargs: (kwargs["m1"] + kwargs["m2"]) ** 2,
    controls=controls,
    c="grey",
    linewidth=0.5,
    label=R"$s_\mathrm{thr}$",
    ax=ax2,
)
iplt.plot(
    plot_domain,
    lambda *args, **kwargs: (
        y_factor * 1j * np_phase_space_factor(*args, **kwargs)
    ).real,
    label="Real part",
    controls=controls,
    ylim=ylim,
    alpha=0.7,
    ax=ax1,
    c="black",
    linestyle="dashed",
)
iplt.plot(
    plot_domain,
    lambda *args, **kwargs: (
        y_factor * 1j * np_phase_space_factor(*args, **kwargs)
    ).imag,
    label="Imag part",
    controls=controls,
    ylim=ylim,
    alpha=0.7,
    ax=ax1,
    c="red",
)

iplt.plot(
    plot_domain,
    lambda *args, **kwargs: y_factor
    * np_chew_mandelstam_s_wave(*args, **kwargs).real,
    label="Real part",
    controls=controls,
    ylim=ylim,
    alpha=0.7,
    ax=ax2,
    c="black",
    linestyle="dashed",
)
iplt.plot(
    plot_domain,
    lambda *args, **kwargs: y_factor
    * np_chew_mandelstam_s_wave(*args, **kwargs).imag,
    label="Imag part",
    controls=controls,
    ylim=ylim,
    alpha=0.7,
    ax=ax2,
    c="red",
)

for ax in axes:
    ax.legend(loc="lower right")
    ax.set_xticks(np.arange(0, 1.21, 0.3))
    ax.set_yticks(np.arange(-1, 1.1, 0.5))
    ax.set_xlabel("$s'$ (GeV$^2$)")

ax1.set_ylabel(R"$16\pi \; i\rho(s)$")
ax2.set_ylabel(R"$16\pi \; \Sigma(s)$")
ax1.set_title(R"Complex phase space factor $\rho$")
ax2.set_title("Chew-Mandelstam $S$-wave ($L=0$)")
plt.show()

### General dispersion integral

For $L>0$ one has to "compute" the following dispersion integral:

$
\qquad \Sigma_a(s+0i) =
    \frac{s-s_{\mathrm{thr}_a}}{\pi}
    \int^\infty_{s_{\mathrm{thr}_a}} \frac{
        \rho_a(s')n_a^2(s')
    }{
        (s' - s_{\mathrm{thr}_a})(s'-s-i0)
    }
    \mathop{}\!\mathrm{d}s'
$

with:
- $\rho$ the normal [`phase_space_factor`](https://ampform.readthedocs.io/en/stable/api/ampform.dynamics.html#ampform.dynamics.phase_space_factor)
- $n_a^2$ the same as [`BlattWeisskopfSquared`](https://ampform.readthedocs.io/en/stable/api/ampform.dynamics.html#ampform.dynamics.BlattWeisskopfSquared)

In [ ]:
def na2(s, m1, m2, L, q0):
    q_squared = breakup_momentum_squared(s, m1, m2)
    return BlattWeisskopfSquared(
        z=q_squared / (q0 ** 2),
        angular_momentum=L,
    )

In [ ]:
q0 = sp.Symbol("q0", real=True)
L = sp.Symbol("L", integer=True, positive=True)
s_thr = (m1 + m2) ** 2
integrand = (
    phase_space_factor(s_prime, m1, m2) * na2(s_prime, m1, m2, L, q0)
) / ((s_prime - s_thr) * (s_prime - s - epsilon * sp.I))

The AmpForm implementation looks like this:

In [ ]:
integrand_subs = integrand.subs(
    {
        2
        * breakup_momentum_squared(s_prime, m1, m2): 2
        * sp.Symbol("q_a^{2}"),
        phase_space_factor(s_prime, m1, m2): sp.Symbol(R"\rho_a(s^{\prime})"),
        s_thr: sp.Symbol(R"s_\mathrm{thr}"),
    }
)
display(Math(f"{sp.latex(integrand_subs)} = {sp.latex(integrand)}"))

and integrate over $s' \in [s_{\mathrm{thr}_a}, \infty)$ with [`scipy.integrate`](https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html) ([QUADPACK](https://en.wikipedia.org/wiki/QUADPACK))

In [ ]:
np_integrand = sp.lambdify(
    args=(s, s_prime, L, epsilon, m1, m2, q0),
    expr=integrand.doit(),
    modules="numpy",
)

s_thr_val = float(s_thr.subs({m1: m1_val, m2: m2_val}))

In [ ]:
def integral(s, L, epsilon, m1, m2, q0):
    values_real, errors_real = scipy.integrate.quad(
        lambda s_prime: np_integrand(s, s_prime, L, epsilon, m1, m2, q0).real,
        a=s_thr_val,
        b=np.inf,
    )
    values_imag, errors_imag = scipy.integrate.quad(
        lambda s_prime: np_integrand(s, s_prime, L, epsilon, m1, m2, q0).imag,
        a=s_thr_val,
        b=np.inf,
    )
    return values_real + values_imag * 1j

In [ ]:
%%time
np_integral = np.vectorize(integral)
s_domain = np.linspace(s_min, s_max, num=50)
max_L = 3
l_values = list(range(1, max_L + 1))
integral_values = {
    L: np_integral(
        s_domain,
        L=L,
        epsilon=1e-3,
        m1=m1_val,
        m2=m2_val,
        q0=1.0,
    )
    for L in l_values
}

sigma = {
    L: (s_domain - s_thr_val) / np.pi * integral_values[L] for L in l_values
}
sigma_scaled = {L: 16 * np.pi * sigma[L] for L in l_values}

### Questions about the dispersion integral

$
\qquad \Sigma_a(s+0i) =
    \frac{s-s_{\mathrm{thr}_a}}{\pi}
    \int^\infty_{s_{\mathrm{thr}_a}} \frac{
        \rho_a(s')n_a^2(s')
    }{
        (s' - s_{\mathrm{thr}_a})(s'-s-i0)
    }
    \mathop{}\!\mathrm{d}s'
$

- What is $q_a$ in [Eq. (49.35)](https://pdg.lbl.gov/2020/reviews/rpp2020-rev-resonances.pdf#page=9) ($S$-wave)?
- How to integrate over $s'$?
- Do the results on the right look as expected?
- Can $\Sigma_a$ be used in a Breit-Wigner as well? <br>
  _See [paragraph under Fig. 49.3](https://pdg.lbl.gov/2020/reviews/rpp2020-rev-resonances.pdf#page=10) and [Eq. (49.24)](https://pdg.lbl.gov/2020/reviews/rpp2020-rev-resonances.pdf#page=7)_
- Are there "Chew-Mandelstam functions" for $L>0$ as well?

In [ ]:
fig, axes = plt.subplots(
    nrows=len(l_values),
    sharex=True,
    figsize=0.6 * np.array([10, 5 * len(l_values)]),
    tight_layout=True,
)
fig.suptitle(f"Dispersion integrals for $m_1={m1_val:.2f}, m_2={m2_val:.2f}$")
for ax, L in zip(axes, l_values):
    ax.axhline(0, linewidth=0.5, c="black")
    ax.axvline(s_thr_val, c="grey", linewidth=0.5, label=R"$s_\mathrm{thr}$")
    ax.plot(
        s_domain,
        sigma_scaled[L].real,
        c="black",
        linestyle="dashed",
        label="Real part",
    )
    ax.plot(
        s_domain,
        sigma_scaled[L].imag,
        c="red",
        label="Imag part",
    )
    ax.set_title(f"$L = {L}$")
    ax.set_ylabel(R"$16\pi \; \Sigma(s)$")
axes[-1].set_xlabel("$s$ (GeV$^2$)")
axes[0].legend()
plt.show()